# Параметры и функции

In [ ]:
from gspread.utils import finditem
#@title Загрузка основных функций и параметров. Авторизация. Запуск обязателен!
#Параметры рассылки
!pip install pillow qrcode #Загрузка библиотеки для генерации QR кодов
!pip install -U transliterate #Загрузка библиотеки для транслитерации
#Имя документа в Google Sheets
#Список всех членов клуба
CLUB_CP_MEMBERS_SHEET  = ''
#Внесённые вручную участники для перечислени бонусов
# MANUAL_ADD_SHEET + mm (номер мксяцев 01...12)
MANUAL_ADD_SHEET = '' 
#SCAN_SHEET таблица сканирования qr на акциях
SCAN_SHEET = ''
# История всех посещенных акций + mm (номер мксяцев 01...12)
EVENTS_VISITS = ''
ev_columns = ['date','phone','name','expire','email','event']


#имена файлов отсётов для ВВ
VV_EVENTS_REPORT = ''
VV_MEMBERS_REPORT = ''

#Отправка только своим - True, всем - False
TEAM_SEND = False
#Мия файла лога
LOG_FILE_NAME = 'mailing.log'
#Путь к google Диску
G_DRIVE_PATH = ''
#пути к csv файлов с отсканированными qr кодами
evet_qr_path = G_DRIVE_PATH + ''
evet_qr_path_archive = G_DRIVE_PATH + ''
new_members_import = G_DRIVE_PATH + ''
new_members_archive = G_DRIVE_PATH + ''
#smtp сервер
SMTP_SERVER = "smtp.yandex.ru"  #"smtp.gmail.com"
SMTP_PORT =   465 # 587 #

#Адрес и пароль для отправки почты
SENDER_EMAIL = "info@good-mapp.com" 
SENDER_PASSWORD = ""                
#Адрес отправителя
FROM = 'info@good-mapp.com'

#Соответсвия называний колонок основной таблицы
COLUMS_NAMES = {
     'date': 'Дата'
    ,'name':'Имя Фамилия'
    ,'phone':'Номер телефона'
    ,'email':'Email'
    ,'note':'Примечания'
    ,'sended':'QR-код отправлен'
    ,'team':'Свои'
    ,'expire':'Срок действия'
    ,'print':'Надо печатать'
    ,'vv_card':'Номер карты ВВ'
}


ABC = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

#Тема письма рассылуа qr
SUBJECT = "Ваш персональный QR-код от проекта Чистый Петербург"
def make_text(name):
  global SUBJECT 
  SUBJECT = "Ваш персональный QR-код от проекта Чистый Петербург"
  html = f"""
  <html>
  <body>
  <p>Добрый день, {name} !</p>

  .....
  
  </body>
  </html>
  
  """
  return html  


def make_text_nocard(name):
  global SUBJECT 
  SUBJECT = "Ждем номер вашей карты ВкусВилл для начисления бонусов"
  html = f"""
  <html>
  <body>
  <p>Добрый день, {name} !</p>

  ...

  
  </body>
  </html>
  
  """
  return html  

def make_text_media(name):
  global SUBJECT 
  SUBJECT = "Раздельный сбор у каждой станции метро в Петербурге"
  html = f"""
  <html>
  <body>
    <p>Уважаемая {name} !</p>

    <p>В Петербурге запустился благотворительный проект “Чистый Петербург” по мобильному раздельному сбору вторсырья.....</p>
  .....
  </body>
  </html>
  
  """
  return html    


#генерация qr  
import qrcode 
from qrcode.image.styledpil import StyledPilImage
from qrcode.image.styles.moduledrawers import RoundedModuleDrawer
from qrcode.image.styles.colormasks import RadialGradiantColorMask
from PIL import Image
#отпраыка e-mail smtp
import smtplib 
#формирование тела письма
from email.mime.text import MIMEText 
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
#кодирования информации в 64-разрядный код (6 бит), 
#широко используемый в приложениях электронной почты
from email import encoders
#BeautifulSoup для парсига html в текст
from bs4 import BeautifulSoup as bs
#для работы с таблицей
import pandas as pd
#для подулючения google drive
from google.colab import drive
from google.colab import auth
#для работы с google sheets
import gspread
from oauth2client.client import GoogleCredentials
#длф работы с датой
from datetime import datetime
import datetime as dt
from transliterate import translit
import calendar

#получение текущего месяца
MONTH = datetime.now().strftime("%m")
YEAR = datetime.now().strftime("%Y")

def correct_phone_number(p_number):
  return p_number[-10:]

def split_delsps(str_in,div=','):
  return str_in.replace(' ', '').split(div)

  
def make_qrcode(row): 
  # taking image which user wants
  # in the QR code center
  logo = G_DRIVE_PATH + '/MyDrive/g-landia/logo.png' 
  back_ground = G_DRIVE_PATH + '/MyDrive/g-landia/back_pic.png'   
  back_ground = Image.open(back_ground)      

  QRcode = qrcode.QRCode(
      error_correction=qrcode.constants.ERROR_CORRECT_H
      ,border=2,
      box_size=14
  )  
  # adding DATA text to QRcode
  data = (f"{split_delsps(row['email'])[0]} {row['expire']} "
          + f"{translit(row['name'], reversed=True)} "
          + f"{correct_phone_number(row['phone'])}"
      )
  QRcode.add_data(data)
  # generating QR code
  QRcode.make() 
  # adding color to QR code
  QRimg = QRcode.make_image(image_factory=StyledPilImage, 
      module_drawer=RoundedModuleDrawer(),
      # color_mask=RadialGradiantColorMask(center_color=(0,255,253),                                         
      #                                    edge_color=(255,0,252)),
      fill_color='Black', 
      back_color="white",      
      embeded_image_path=logo).convert('RGB')
  pos = ((back_ground.size[0] - QRimg.size[0]) // 2,
        (back_ground.size[1] - QRimg.size[1] - 15) // 2)
  back_ground.paste(QRimg, pos)
  filename = "qr_code.png"
  # save the QR code generated
  back_ground.save(filename)
  return filename


def make_mail(text_to_send, to='', files_to_send='no_file', 
               from_email=FROM):
  # initialize the message we wanna send
  subject=SUBJECT
  msg = MIMEMultipart("mixed")
  msg["From"] = from_email
  msg["To"] = to
  msg["Subject"] = subject
  message = MIMEMultipart("alternative")
  html = text_to_send
  text = bs(html, "html.parser").text
  message.attach(MIMEText(text, 'plain'))
  message.attach(MIMEText(html, 'html'))
  msg.attach(message)
  if files_to_send != 'no_file':
    part = MIMEBase("image", "png")
    with open(files_to_send, "rb") as attachment:    
      part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 
                   f"attachment; filename={files_to_send}")
    msg.attach(part)
  return msg


def do_mailing(mail_list, team=False, to='', sender=SENDER_EMAIL, 
               password=SENDER_PASSWORD, from_email=FROM, msg_text='main'):
  # initialize the SMTP server
  server = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT)
  # connect to the SMTP server as TLS mode (secure) and send EHLO
  # server.starttls()
  # login to the account using the credentials
  server.login(sender, password)
  log = []
  log_str = ''
  for index, row in mail_list.iterrows():
    if (not team) or row['team'] == 'ДА':
      if ((row['email'] not in ['','-'] and row['sended'] == '')
          or msg_text =='no card') :
        try:          
          if msg_text == 'main':
            qr_code_file = make_qrcode(row)
            msg = make_mail(make_text(row['name']), to=row['email'], 
                            files_to_send=qr_code_file, from_email=from_email)
          elif msg_text == 'no card':
            msg = make_mail(make_text_nocard(row['name']), to=row['email']
                            , from_email=from_email)
          elif msg_text == 'media':
            msg = make_mail(make_text_media(row['name']), to=row['email']
                            , from_email=from_email)

          # send the email
          server.sendmail(from_email, split_delsps(row['email']), msg.as_string())
          log_str = (f"{datetime.now()} {index} || {row['name']} || "
                  + f"{row['phone']} || {row['email']} || Email has been sent"
                  )
          print(log_str)
          log.append(log_str)
          row['sended'] = 'ДА'
        # except ValueError:
        except BaseException as err:
          log_str = (f"{datetime.now()} {index} || {row['name']} || "
                  + f"{row['phone']} || {row['email']} ||Err {err}, {type(err)}"
                  )
          print(log_str)
          log.append(log_str)
  # terminate the SMTP session
  server.quit()
  
  with open(LOG_FILE_NAME, 'a') as f:
    f.write(str(log)+'\n')
  return mail_list


def get_keybyval(val, dc=COLUMS_NAMES):
    """
    key by value in dict
    """    
    for idn, v in dc.items():
        if v == val:
            return(idn)
    return val   


def rename_colbyvals(data, vals, dc=COLUMS_NAMES):
    """
    rename columns by list of vals
    using columns name dict
    """
    for val in vals:
        data = data.rename(columns={val:get_keybyval(val, dc)})
    return data


def get_abcrage(x1, y1, x2, y2):
  return f"{ABC[x1]}{y1}:{ABC[x2]}{y2}"


def all_memb_rewrite_column(data, column_name, sheet, start_str=2):
  #установка статуса отправки писем в google sheets
  n_count = len(data)+1
  n_columns = len(data.columns)
  qrc_s_ind = 0
  for i in range(n_columns):
    if data.columns[i] == column_name:
      qrc_s_ind = i
  range_s = get_abcrage(qrc_s_ind, start_str, qrc_s_ind, n_count)
  status = sheet.range(range_s)
  for ind, row in data.iterrows():
    status[ind].value = row[column_name]
  sheet.update_cells(status)


def do_mailing_and_log(mail_list, sheets):  
  mail_list = do_mailing(mail_list, team=TEAM_SEND)

  #Перенос log файла на google диск
  LOG_PATH = G_DRIVE_PATH+'/MyDrive/'+LOG_FILE_NAME
  %mv $LOG_FILE_NAME $LOG_PATH
  all_memb_rewrite_column(mail_list, 'sended', sheets)


def get_all_values_sheets(file_name):
  try:
    sheets = g_sheets.open(file_name)
  except:
    print(f'Google таблица: "{file_name}" не найдена')
    return None, None
  return  sheets, sheets.sheet1.get_all_values()


def get_all_memb_df(rows): 
  data_memb = pd.DataFrame(rows[1:], columns=rows[0])
  data_memb = rename_colbyvals(data_memb,data_memb.columns)
  return data_memb

def get_all_memb_fr_sheet(sheets): 
  return get_all_memb_df(sheets.sheet1.get_all_values())
  
  
def move_files(file_list, src, dest):
  # перенос csv файлов в архив
  for file in file_list:  
    if '.csv' in file: 
      source = src+file
      destination = dest+file
      !mv $source $destination


def make_vvrep(data, rep_cols=['date', 'vv_card', 'phone']):  
  data = data[rep_cols].copy()
  if 'date' in rep_cols:
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
    data['date'] = data['date'].apply(lambda x:x.strftime('%d.%m'))

  data.drop_duplicates(inplace=True)
  data.reset_index(drop=True, inplace=True)
  return data


def vvrep_from_gs(file_name, columns):
  sheets, rows = get_all_values_sheets(file_name+MONTH)
  if rows == None: 
    return None
  data = make_vvrep(pd.DataFrame(rows, columns=columns))
  if len(data) > 0:
    list_name = (f"{data['date'].min().split('.')[0]}-"
                + f"{data['date'].max()}"
              )    
    sheets.sheet1.update_title(list_name)
    sheets.add_worksheet(title="new", rows="37", cols="10", index=0) 
  return data  

  
def get_data_from_gs(sheets, rows, columns, new_sheet=True):
  if rows == None: 
    return None
  data = pd.DataFrame(rows, columns=columns)
  if len(data) > 0:
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
    data_range = data['date'].apply(lambda x:x.strftime('%d.%m'))
    list_name = (f"{data_range.min().split('.')[0]}-"
                + f"{data_range.max()}"
              )    
    sheets.sheet1.update_title(list_name)
    if new_sheet:
      sheets.add_worksheet(title="new", rows="37", cols="10", index=0) 
      #добавление столбцов на новый лист
      cl_rng = get_abcrage(0, 1, len(columns) - 1, 1)   
      cel_l = sheets.sheet1.range(cl_rng)
      for i in range(len(columns)):      
        cel_l[i].value = columns[i]
      sheets.sheet1.update_cells(cel_l)
  return data  


def get_file_list(path, ext='csv', echo=True):
  """
  Получение списка файлов  
  """
  finde = f".{ext}"  
  file_list = !ls $path | grep $finde  
  print(f"В дирректории {path} найдено {ext} файлов {len(file_list)}")
  if len(file_list) > 0:
    print(f"{file_list}")
  return file_list


def move_files(file_list, source, destination, ext='csv', echo=False):
  """
  Перенос csv файлов в архив
  """
  for file in file_list:  
    if ext in file:       
      sr_file = source+file
      dest_file = destination+file
      if echo:
        print(f"Перемеще {sr_file} -> {dest_file}")
      !mv $sr_file $dest_file


#подключение Google диска
drive.mount(G_DRIVE_PATH)
!mkdir $evet_qr_path
!mkdir $evet_qr_path_archive
!mkdir $new_members_import
!mkdir $new_members_archive
#Аутентификация Google аккаунта
auth.authenticate_user()
#Досуп к аккаунту дял библиатки gspread
g_sheets = gspread.authorize(GoogleCredentials.get_application_default())

#Открыывем таблицу
all_members_sheets, all_members = get_all_values_sheets(CLUB_CP_MEMBERS_SHEET)
all_members = get_all_memb_df(all_members)


# Рассыка для СМИ

In [ ]:
#@title Рассылка для СМИ
MEDIA_MAIL_SHEET = ''
media_mail_list_sheets, media_mail_list = get_all_values_sheets(MEDIA_MAIL_SHEET)
media_mail_list = pd.DataFrame(media_mail_list[1:], columns=['name','email','sended','phone'])
# display(media_mail_list)

media_sender = ""
media_pass = "" 

mail_list_media = do_mailing(media_mail_list, sender=media_sender, password=media_pass, 
                      from_email=media_sender, msg_text='media')
all_memb_rewrite_column(mail_list_media, 'sended', media_mail_list_sheets.sheet1)

# Создание отчётов

In [ ]:
#@title Рассылка для тех у кого нет карты
nocard_mail_list = (all_members[all_members['vv_card'] == 'Нет карты']
                    .reset_index(drop=True)
                  )
display(nocard_mail_list)
# display(nocard_mail_list)
mail_list = do_mailing(nocard_mail_list, msg_text='no card')

In [ ]:
#@title Импорт номеров карт
CardsVV_file = ''
cards = pd.DataFrame
sheets, rows = get_all_values_sheets(CardsVV_file)
cards = pd.DataFrame(rows, columns = ['phone', 'vv_card', 'amaunt', 'desc'])

# cards = cards.loc[cards['phone'] ]
cards['phone'] = cards['phone'].str.extract(r'(\d{10})')
cards = cards[cards['phone'].notna()]
no_cards = cards[cards['vv_card'] == '']['phone']
cards = cards[cards['vv_card'] != ''][['phone','vv_card']]
no_cards.drop_duplicates(inplace=True)
no_cards.reset_index(drop=True, inplace=True)
cards.drop_duplicates(inplace=True)
cards.reset_index(drop=True, inplace=True)
# display(cards)
# display(no_cards)


def get_vv_card(row):
  vv_card = cards[cards['phone'] == row['phone']]['vv_card'].values
  try:
    vv_card = vv_card[0]
  except:
    vv_card = 'Нет карты'  
  return vv_card


all_members['vv_card'] = all_members.apply(get_vv_card, axis=1)
memb_nocard = all_members[all_members['vv_card'] == 'Нет карты'].reset_index(drop=True)

all_memb_rewrite_column(all_members, 'vv_card', all_members_sheets.sheet1)

# not_memb = []
# for phone in no_cards:
#   if phone not in memb_nocard['phone'].values:
#     not_memb.append(phone)
# display(not_memb)

no_card_fName = 'Цлены клуба без карты'
try: 
  no_card_st = g_sheets.open(no_card_fName).sheet1
except:
  no_card_st = g_sheets.create(no_card_fName).sheet1

#получение номера последней записи в таблице 
row_cnt, col_cnt = memb_nocard.shape

#добавление данных в гугл таблицу
cl_rng = get_abcrage(0, 1,
                     col_cnt - 1, row_cnt + 1)
cel_l = no_card_st.range(cl_rng)
ind = 0
for i in range(row_cnt):
  for j in range(col_cnt):
    cel_l[ind].value = memb_nocard.iloc[i][j]
    ind += 1
no_card_st.update_cells(cel_l)



In [ ]:
#@title Функции для отчётов, запускать перед созданием отчётов
from ast import Str
def vvrep_to_gs(file_name, data):  
  try: 
    sheet = g_sheets.open(file_name+MONTH).sheet1
  except:
    sheet = g_sheets.create(file_name+MONTH).sheet1
  sheet.clear()
  #добавление данных в гугл таблицу
  row_cnt, col_cnt = data.shape  
  # display(data)
  cl_rng = get_abcrage(0, 1, col_cnt - 1, row_cnt)
  # print(cl_rng)
  cel_l = sheet.range(cl_rng)
  ind = 0
  for i in range(row_cnt):
    for j in range(col_cnt):
      cel_l[ind].value = str(data.iloc[i][j])
      ind += 1
  sheet.update_cells(cel_l)

  # for ind, row in data.iterrows():    
  #   sheet.insert_row(list(row),ind+1)

def add_team_memb(data, start_day=1):
  team = (all_members[(all_members['team'] == 'ДА') 
                    & (all_members['vv_card'] != 'Нет карты')][['vv_card','phone']]
  )
  team['count'] = 3
  # last_day = calendar.monthrange(int(YEAR), int(MONTH))[1]
  # team_bonus = []
  # for team_memb in team:
  #   for i in range(start_day, last_day):
  #     if (i % 3) == 0:
  #       n_day = str(i)
  #       if len(n_day) == 1:
  #         n_day = '0'+ n_day
  #       team_bonus.append([f"{n_day}.{MONTH}", team_memb])  
  # team_bonus = pd.DataFrame(team_bonus, columns=['date', 'phone'])    

  return pd.concat([data, team]).sort_values(by='count').reset_index(drop=True)

def get_vv_card_all(row):
  vv_card = all_members[all_members['phone'] == row['phone']]['vv_card'].values  
  try:
    vv_card = vv_card[0]
  except:
    vv_card = 'Не в клубе'  
  return vv_card

In [ ]:
#@title Месячеый отчёт о членах клуба 
#VV_MEMBERS_REPORT
all_members = get_all_memb_fr_sheet(all_members_sheets)
vv_rep_members = make_vvrep(all_members[all_members['vv_card']!='Нет карты'],['vv_card'])
vvrep_to_gs(VV_MEMBERS_REPORT, vv_rep_members)

In [ ]:
#@title Отчёт - Чистый Петербург акции (+месяц) создаются в Гугл таблицах 
#VV_EVENTS_REPORT
#отчёт посещений акций
vv_rep_events = vvrep_from_gs(EVENTS_VISITS, ev_columns+['vv_card'])
if (type(vv_rep_events) != None and len(vv_rep_events)):  
  vv_rep_events = (vv_rep_events.groupby(['vv_card','phone']).count()
                                .rename(columns={'date':'count'})
                                .sort_values(by='count', ascending=False)
                                .reset_index(drop=False)
                  )
  vv_rep_events = add_team_memb(vv_rep_events[['vv_card','phone','count']])   
  vv_rep_events = (vv_rep_events.groupby(['vv_card','phone']).sum()
                                .rename(columns={'date':'count'})
                                .sort_values(by='count', ascending=False)
                                .reset_index(drop=False)
    )
  
  # display(vv_rep_events)
  # vv_rep_events['vv_card'] = vv_rep_events.apply(get_vv_card_all, axis=1)
  # vv_rep_events_no_send = vv_rep_events[vv_rep_events['vv_card'].isin(['Нет карты','Не в клубе'])]
  # vv_rep_events_send = vv_rep_events[~vv_rep_events['vv_card'].isin(['Нет карты','Не в клубе'])]
  # display(vv_rep_events_no_send.sort_values(by='vv_card'))
  vvrep_to_gs(VV_EVENTS_REPORT, vv_rep_events[['vv_card','count']])

# Рассылка QR кодов

In [ ]:
#@title Импорт новых членов клуба

NEW_MEMBERS_COLUMNS = {
'id': 'id', 
'name': 'Имя донора', 
'email': 'Email', 
'payment_type': 'Тип платежа', 
'payment_oper': 'Плат. оператор',
'payment_method':'Способ платежа', 
'full_amount': 'Полная сумма', 
'total_amount': 'Итоговая сумма', 
'Currency': 'Currency',
'date': 'Дата пожертвования', 
'status': 'Статус', 
'campaign': 'Кампания', 
'purpose': 'Назначение',
'subscribe': 'Подписка на рассылку', 
'email_sub': 'Email подписки', 
'comment': 'Комментарий'
}


data_new_members = pd.DataFrame()

def get_new_members(file_name):
  """
  Чтение и обработка данных из cvs файла
  """
  data_memb = pd.DataFrame()
  data_memb = pd.read_csv(file_name, header=0, index_col=False,
                          sep='\t',
                          encoding ='UTF-16LE'
                          )
  
  data_memb = rename_colbyvals(data_memb, data_memb.columns, 
                            dc=NEW_MEMBERS_COLUMNS)
  data_memb = data_memb.loc[data_memb['status'] == 'Оплачено']  
  data_memb['name'] = data_memb['name'].apply(lambda  x: x.split()[0])

  return data_memb[['date','name','email','full_amount']]

def get_new_memb_allfiles(columns):
  #получение списка файлов  
  data_memb = pd.DataFrame()
  file_list = !ls $new_members_import
  file_list = file_list[0].split()
  files_cnt = 0  
  # if len()
  for file in file_list:
    #сбор данных из cvs фпйлов
    if '.csv' in file: 
      files_cnt += 1
      data_memb = pd.concat([data_memb, 
                           get_new_members(new_members_import + file)], 
                          ignore_index=True
                          )
      
      data_memb = (data_memb.groupby('email')
                              .agg({'date':'last',
                                    'name':'last', 
                                    'full_amount':'sum'})
                              .reset_index(drop=False)
                )
  if files_cnt > 0:      
    data_memb = data_memb[data_memb['full_amount'] >= 2000]                                    
    data_memb['date'] = pd.to_datetime(data_memb['date'])
    live_time = dt.timedelta(days=335)
    data_memb['expire'] = (pd.DatetimeIndex(data_memb['date']+live_time)
                              .strftime("%m.%Y")
                          )
    data_memb['phone'] = '!!!'
    data_memb['note'] = ''
    data_memb['qrc_status'] = ''
    data_memb['team'] = ''
    data_memb['print'] = ''
  
    move_files(file_list, new_members_import, new_members_archive)
  else:
    print(f'Данные для импорта не найдены')
    return pd.DataFrame(columns=columns)
  return data_memb[columns].reset_index(drop=True)

memb_columns = ['name','phone','email','note','qrc_status',
                'team','expire','print']

data_new_members = get_new_memb_allfiles(memb_columns)
all_mamb_email = list(all_members[~all_members['email']
                                  .isin(['','-'])]['email']
                      )
data_new_members = (data_new_members[~data_new_members['email']
                         .isin(all_mamb_email)]
                      )

#получение номера последней записи в таблице 
mamb_last_row = len(all_members_sheets.sheet1.get_all_values())
row_cnt, col_cnt = data_new_members.shape

#добавление данных в гугл таблицу
cl_rng = get_abcrage(0, mamb_last_row + 1,
                     col_cnt - 1, mamb_last_row + row_cnt + 2)
# print(cl_rng)
cel_l = all_members_sheets.sheet1.range(cl_rng)
ind = 0
for i in range(row_cnt):
  for j in range(col_cnt):
    cel_l[ind].value = data_new_members.iloc[i][j]
    ind += 1
all_members_sheets.sheet1.update_cells(cel_l)

all_members = get_all_memb_fr_sheet(all_members_sheets)


In [ ]:
#@title Запустить для Рассылки QR кодов
all_members = get_all_memb_fr_sheet(all_members_sheets)
do_mailing_and_log(all_members, all_members_sheets.sheet1)

# Посещение акций

In [ ]:
from numpy.core.numeric import NaN
#@title Импорт QR кодов из csv и google sheets


def fix_qr_data(arr):
  """
  Обработка и коррекция данных из qr - кодов
  """
  arr = arr.split(' ')
  result = []
  for a in reversed(arr):
    if a not in ['','декабрь','2022']:
      result.append(a)
  return result


def check_field(row, field_name):
  """
  Функция для заполнения пустых полей
  """
  if (field_name not in row.index) or (row[field_name] == ''):    
    result = (all_members.loc[(all_members['phone'] == row['phone']) 
                          | (all_members['vv_card'] == row['vv_card'])
                          , [field_name]].values
          )
    if len(result) > 0:      
      row[field_name] = result[0][0]
    else:
      row[field_name] = f"{COLUMS_NAMES[field_name]} не указано"
  return row[field_name] 


def fill_data(data):    
  """
  Функция для востановления двнных из разных источников
  """  
  data_ev = pd.DataFrame(columns=ev_columns)  
  # if 'event' not in data.columns:
  #   data['event'] = data['gps']       
  if 'qr_data' in data.columns: #разбор строки qr кода    
    for ind, row in data.iterrows():
      data_str = pd.Series([],dtype=pd.StringDtype())
      data_str['date'] = row['date']      
      if 'gps' in data.columns:
        data_str['event'] = row['gps']      
      elif ('gps_a' in data.columns) and ('gps_b' in data.columns):
        data_str['event'] = f"{row['gps_a']},{row['gps_b']}"
      str_datd = fix_qr_data(row['qr_data'])    
      for i in range(len(str_datd)): #расперд строки qr кода по колонкам
        data_str[ev_columns[1 + i]] = str_datd[i]    
      data_ev = data_ev.append(data_str, ignore_index=True)
    data_ev['vv_card'] = ''  
  else:
    data_ev = data
  data_ev.fillna('', inplace=True)
  for ind, row in data_ev.iterrows(): #Заполнение недостающих полей
    for col in ev_columns+['vv_card']:
      if col not in ['event']:    
        data_ev.loc[ind,col] = check_field(row, col)
  data_ev.fillna('', inplace=True)
  data_ev['date'] = pd.to_datetime(data_ev['date'])    
  return data_ev


def get_events_qr(file_name):
  """
  Чтение и обработка данных из cvs файла
  """  
  data = pd.read_csv(file_name, header=None, names=['date','qr_data','gps']
                     , index_col=False)  
  return fill_data(data)

#Импорт данных с акций
data_events = pd.DataFrame(columns=ev_columns)

#Импорт данных опосещениях из CSV файлов
file_list = get_file_list(evet_qr_path)
for file in file_list:      
  data_events = pd.concat([data_events, 
                           get_events_qr(evet_qr_path + file)], 
                          ignore_index=True
                          )
move_files(file_list, evet_qr_path, evet_qr_path_archive)

#Импорт данных об участниках введйнных вручную
ma_sheets, rows = get_all_values_sheets(MANUAL_ADD_SHEET+MONTH)
if ma_sheets != None and len(rows) > 1:  
  ma_bonus =  get_data_from_gs(ma_sheets, rows[1:]
                                , columns=rows[0]
                                ,  new_sheet=True)    
  ma_bonus['event'] = 'Бонус за вступление, ручной ввод'  
  #заполнение полей
  ma_bonus = fill_data(ma_bonus)  
  #Добавления данных введйнных вручную к остальным
  data_events = pd.concat([data_events, ma_bonus], ignore_index=True)
else:
  print(f"Данные в таблтце \"{MANUAL_ADD_SHEET+MONTH}\" не найдены")  

#импорт данных из таблици сканировани
scan_ev = pd.DataFrame(columns=ev_columns)
scan_sheets, scan_rows = get_all_values_sheets(SCAN_SHEET)
if scan_sheets != None and len(scan_rows) > 1:
  scan_ev = get_data_from_gs(scan_sheets, scan_rows[1:]
                                , columns=scan_rows[0]
                                , new_sheet=False)    
  #заполнение полей
  scan_ev = fill_data(scan_ev)   
  #Добавления данных сканированных qr к остальным
  data_events = pd.concat([data_events, scan_ev], ignore_index=True)
else:
  print(f"Данные в таблтце \"{SCAN_SHEET}\" не найдены")  


#===================Обработка данных====================
#сортировка и заполненин nan
data_events.sort_values(by='date', inplace=True)
data_events.reset_index(drop=True, inplace=True)
data_events.fillna('', inplace=True)

#создание имени файла с номером месяца
events_file_n = f"{EVENTS_VISITS}{MONTH}"
#создание или открытие гугл таблицы 
try: 
  events_log = g_sheets.open(events_file_n).sheet1
except:
  events_log = g_sheets.create(events_file_n).sheet1  
#получение номера последней записи в таблице 
last_row = len(events_log.get_all_values())
row_cnt, col_cnt = data_events.shape

#добавление данных в гугл таблицу
cl_rng = get_abcrage(0, last_row + 1, col_cnt - 1, last_row + row_cnt + 2)
# print(cl_rng)
cel_l = events_log.range(cl_rng)
ind = 0
for i in range(row_cnt):
  for j in range(col_cnt):
    cel_l[ind].value = str(data_events.iloc[i][j])
    ind += 1
events_log.update_cells(cel_l) 